In [139]:
#verificar instalação do selenium
#utilizar o comando !pip install selenium -i http://binario.caixa:8081/repository/pypi-repo/simple --trusted-host binario.caixa --user
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import os
import getpass
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver.common.by import By
import pandas as pd
import sqlite3

from dotenv import load_dotenv
from dotenv import find_dotenv
load_dotenv(find_dotenv())

True

In [140]:
df_base = None
def login_atender(matricula, senha):
    browser = webdriver.Firefox(executable_path='geckodriver.exe')

    # abrir a página do Atender.caixa
    url = 'https://atender.caixa/siouv/jsp/login/login.jsp'

    browser.get(url)

    form_usuario = browser.find_element_by_name('j_username')
    form_usuario.send_keys(matricula)

    form_senha = browser.find_element_by_name('j_password')
    form_senha.send_keys(senha)

    botao_login = browser.find_element_by_xpath('//form/table/tbody/tr/td/table[1]/tbody/tr[1]/td/div/table/tbody/tr[4]/td/table/tbody/tr/td[2]/a')
    botao_login.click()

    return browser

In [141]:
# main
matricula = input('Digite sua matrícula: ')
senha = getpass.getpass('Digite sua senha: ')

browser = login_atender(matricula, senha)

Digite sua matrícula: c098453
Digite sua senha: ········


In [142]:
sleep(3)
browser.find_element_by_link_text('Consulta Ocorrências para Tratamento').click()
sleep(3)

In [143]:
html = browser.page_source

In [144]:
df = pd.read_html(html)[0]
df = df[df[3] == "Enviada"]
lista_siouv = df[0].values.tolist()
lista_siouv

['10965805', '10974581', '10990146', '10983911']

In [145]:
for siouv in lista_siouv:
    urlsiouv = '//a[contains(@href,'+ "'"+ siouv + "'"+ ')'+']'
    browser.find_element_by_xpath(urlsiouv).click()
    sleep(3)
    html = browser.page_source
    df = pd.read_html(html)[0]
    df = df[df[1].notna()]
    df = df.iloc[6:]
    soup = BeautifulSoup(html, 'html.parser')
    resultado=soup.find_all(id='tblPesquisarProponente')
    if df[0].str.contains('Protocolo SAC 1111111111111111').any():
        sac = 'Sim'
    else:
        sac =  'Não'
    df = df[~df[0].isin(['Protocolo SAC 1111111111111111'])]
    df = df[~df[0].str.startswith('Nº Pré-Ocorrência')]
    
    #pega natureza
    natureza = soup.find_all('option', {'selected': True})[1].text
    
    #pega unidade
    unidade = soup.find('input', {'name': 'unidadeEnvolvida'}).get('value')
    una = unidade.split(' ')
    
    #pega assunto
    assunto = soup.find_all('option', {'selected': True})[2].text
    
    #pega assunto
    Item = soup.find_all('option', {'selected': True})[3].text
    
    #pega Motivo
    Motivo = soup.find_all('option', {'selected': True})[4].text

    #pega vencimento
    vencimento = df[0].values[0].split(' ')


    #pega data da abertura
    dataAbertura = df[0].values[1].split(' ')


    #pega CPF
    cpf  = df[0].values[2].split(' ')


    #pega Nome
    nome = df[0].values[2].split('-')


    #pega Contato
    contato = df[0].values[3]


    #pega Manifesto
    manifesto = df[1].values[10]


    #pega Comentário
    comentario = df[1].values[11]


    tabela = pd.DataFrame({'numeroSiouv': siouv, 'sac': sac, 'natureza': natureza, 'vencimento': vencimento[1], 'dataAbertura': dataAbertura[2], 'Nome': nome[2]
                          ,'CPF': cpf[2],'contato': contato, 'assunto': assunto, 'item': Item, 'motivo': Motivo,
                           'manifesto': manifesto,'comentario': comentario,'unidade': una[0]},index=[0])
    if(df_base is None):
        df_base = tabela
    else:
        df_base = pd.concat([df_base, tabela], axis=0, join='outer', ignore_index=True)
    browser.get('https://atender.caixa/siouv/jsp/login/ConsultarOcorrencia.do?method=listarOcorrenciasExternasInternas&amp;perfilUsuario=administrador&amp;escopo=I&amp;ordenacaoOuvidoria=1&amp;ordenacaoSac=1&amp;ordenacaoInterna=1&amp;tipoOrdenacaoOuvidoria=ASC&amp;tipoOrdenacaoSac=ASC&amp;tipoOrdenacaoInterna=ASC')
print("FIM")

FIM


In [146]:
df_base

,numeroSiouv,sac,natureza,vencimento,dataAbertura,Nome,CPF,contato,assunto,item,motivo,manifesto,comentario,unidade
0,10965805,Sim,RECLAMAÇÃO,09/10/2020,02/10/2020,Arthur Escorza Glugoski,332.132.938-73,Telefone (11) __9844-45342 / (11__) 9844-4534...,HABITAÇÃO,FINANCIAMENTO HABITACIONAL,LIBERAÇÃO DO FINANCIAMENTO,Categoria: Outros ServiçosNatureza: RECLAMAÇÃO...,"""CLASSIFICAÇÃO APROXIMADA POR NÃO EXISTIR CATE...",7257
1,10974581,Sim,RECLAMAÇÃO,13/10/2020,05/10/2020,joao batista maia da silva,268.873.628-05,Telefone (11) 82564769 / (11) 68366389 / (11)...,HABITAÇÃO,FINANCIAMENTO HABITACIONAL,RENEGOCIAÇÃO DE DÍVIDA,Categoria: Financiamento HabitacionalNatureza:...,CLASSIFICAÇÃO APROXIMADA POR NÃO EXISTIR CATEG...,7257
2,10990146,Sim,RECLAMAÇÃO,15/10/2020,07/10/2020,Alan martins tomaz,299.487.058-40,Telefone (11) 9765-85871 Correio Eletrônico ...,HABITAÇÃO,VENDA DE IMÓVEIS,LEILÃO - IMÓVEIS CAIXA,Categoria: Outros ServiçosNatureza: RECLAMAÇÃO...,“CLASSIFICAÇÃO APROXIMADA POR NÃO EXISTIR CATE...,7257
3,10983911,Não,RECLAMAÇÃO,08/10/2020,06/10/2020,Ligia Colonhesi Berenguel,260.226.228-59,Telefone (11) 9707-15005 Correio Eletrônico ...,HABITAÇÃO,VENDA DE IMÓVEIS,LEILÃO - IMÓVEIS CAIXA,DADOS PESSOAIS Endereço: Telefone: 11970715005...,1. Favor anexar informações a respeito da recl...,7257
